# Clean the raw data
# Merge the preprocessed external data with raw data
# Create fist_preprocessed data csv files for later usage

In [6]:
import pandas as pd
from datetime import datetime

# read in the preprocessed covid, comfort index data, and taxi zone number data
pd.set_option("display.max_columns", None)
covid_data = pd.read_csv(r"../raw_data/raw/covid.csv", index_col = False, delimiter = ',', header = 0, low_memory = False)
district_id = pd.read_csv(r"../raw_data/raw/taxi+_zone_lookup.csv", index_col = False, delimiter = ',', header = 0, low_memory = False)
comfort_index_data = pd.read_csv(r"../raw_data/raw/comfortIndex.csv", index_col = False, delimiter = ',', header = 0, low_memory = False)

# Remove zone 264, 265 as they are unknown zone which cannot be investtigated in geo-spatial visualisation
max_id = max(district_id['LocationID']) - 2
min_id = min(district_id['LocationID'])

# Set minimum trip distance, passenger, fare & other payment (exclude fare & total amount) as 0
min_distance = 0
min_passenger = 0
min_money = 0

# Set minimum fare & total amount as 2.5 
# this value is provided by https://www1.nyc.gov/site/tlc/passengers/taxi-fare.page , Standard Metered Fare section
min_fare_total = 2.5

# Set the year-month stirng value for reading in raw yellow taxi record data
# List of collection of months with no daily covid case (the record of covid case start from 2020.3.1 in New York)
no_covid_date = ["2019-12", "2020-01", "2020-02"]

# List of collection of months with daily covid case
exist_covid_date = ["2020-03", "2020-04", "2020-05", "2020-06", "2020-07", "2020-08", "2020-09", "2020-10", "2020-11"]

# Set the initial timestamp string for each month for later usage (drop instances which don't belong to each csv)
initial_time = "-01 00:00:00"

# Set read in csv file string for function and for loop usage to reduce redundent code
yellow_format = "../raw_data/raw/yellow_tripdata_{year}.csv"
yellow_new_format = "../raw_data/first_preprocess/yellow_{year}.csv"



print("finish")

finish


# Read in taxi data (no-covid), Merge with external data, Create new csv

In [7]:
def yellow_no_covid_process(date, initial_time, format_yellow, yellow_new_format, max_id, min_id, min_money, min_fare_total, min_distance, min_passenger, comfort_index_data):
    '''
    This function is created to:
    Read in yellow taxi data
    
    Drop the useless columns (not related to tip_amount in the assumption)
    Drop the instances with irrational feature values (eg. trip distance < 0)
    
    Create column for recording each trip's duration (feature engineering with dropoff time - pickup time)
    Create columns to record daily covid count (in these months, covid_count = 0) and comfort_index
    
    Merge the taxi data with external datasets (covid & comfort_index)
    
    Create the first_preprocessed csv files for further usage
    '''
    
    # set the initial timestap string for each read in month
    month_start = date + initial_time
    
    # read in the selected month data
    yellow_data = pd.read_csv(format_yellow.format(year = date), index_col=False, delimiter = ',', header = 0, low_memory = False)
    
    
    # drop the columns which are assumed to be irrelavant to tip_amount (target)
    yellow_data = yellow_data.drop(columns = 'store_and_fwd_flag')
    yellow_data = yellow_data.drop(columns = 'RatecodeID')
    yellow_data = yellow_data.drop(columns = 'VendorID')
    
    # drop the irrational instance with .loc() method with listed conditions
    yellow_data = yellow_data.loc[(yellow_data['passenger_count'] > min_passenger) &
                                  (yellow_data['PULocationID'] <= max_id) &
                                  (yellow_data['PULocationID'] >= min_id) &
                                  (yellow_data['DOLocationID'] <= max_id) &
                                  (yellow_data['DOLocationID'] >= min_id) &
                                  (yellow_data['trip_distance'] > min_distance) &
                                  (yellow_data['fare_amount'] >= min_fare_total) &
                                  (yellow_data['extra'] >= min_money) &
                                  (yellow_data['mta_tax'] >= min_money) &
                                  (yellow_data['tip_amount'] >= min_money) &
                                  (yellow_data['tolls_amount'] >= min_money) &
                                  (yellow_data['improvement_surcharge'] >= min_money) &
                                  (yellow_data['total_amount'] >= min_fare_total) &
                                  (yellow_data['congestion_surcharge'] >= min_money)]

    
    # drop irrational instances base on pickup time (must be greater or equal to initial timestamp of this month)
    yellow_data = yellow_data.loc[(pd.to_datetime(yellow_data['tpep_pickup_datetime']) >= datetime.strptime(month_start, '%Y-%m-%d %H:%M:%S'))]

    # create duration column to record trip's duration from the difference between dropoff and pickup time 
    yellow_data['duration'] = ((pd.to_datetime(yellow_data.tpep_dropoff_datetime) - pd.to_datetime(yellow_data.tpep_pickup_datetime)).dt.days * 60*24 + (pd.to_datetime(yellow_data.tpep_dropoff_datetime) - pd.to_datetime(yellow_data.tpep_pickup_datetime)).dt.seconds / 60).round(5)
    
    # drop irrational instances base on the difference between pickup (earlier) and dropoff (later) conditions
    yellow_data = yellow_data.loc[(yellow_data['duration'] > 0)]
    
    yellow_data = yellow_data.sort_values(by = ['tpep_pickup_datetime'])
    
    
    # create column for covid daily count for this month's data (no-covid record as 0)
    yellow_data['covid_count'] = 0

    # create the list for collecting date (yyyy-mm-dd) for this month to selected time period for external data
    month_di = []
    month_value = []
    for i in range(len(comfort_index_data['date'])):
        if date in comfort_index_data['date'][i]:
            month_di.append(comfort_index_data['date'][i])
            month_value.append(comfort_index_data['DIcomfort_index'][i])
    
    # create list for loading comfort_index for this month taxi's data for constructing dataframe's column
    index = 0
    month_comfort_index = []
    pickup_time = yellow_data['tpep_pickup_datetime'].tolist()
    for i in range(len(pickup_time)):
        if (month_di[index] in pickup_time[i]):
            month_comfort_index.append(month_value[index])
        else:
            if (index + 1 < len(month_di)):
                index += 1
                month_comfort_index.append(month_value[index])
            else:
                month_comfort_index.append(-9999)
    
    
    # merge the created comfort_index column with the taxi data
    comfort_column = pd.Series(month_comfort_index)
    yellow_data['comfort_index'] = comfort_column.values
    
    # remove irrational instances base on comfort_index data
    yellow_data = yellow_data.loc[(yellow_data['comfort_index'] > -9999)]
    
    # remove columns of pickup, dropoff columns
    yellow_data = yellow_data.drop(columns = 'tpep_pickup_datetime')
    yellow_data = yellow_data.drop(columns = 'tpep_dropoff_datetime')
    
    # create and store the first_processed csv data under folder 'first_preprocess' in 'raw_data' for futher usage
    yellow_data.to_csv(yellow_new_format.format(year = date), index = False)

In [8]:
# Use for loop to read in no-covid taxi data and create first_preprocessed merged data for further usage
for i in no_covid_date:
     yellow_no_covid_process(i, initial_time, yellow_format, yellow_new_format, max_id, min_id, min_money, min_fare_total, min_distance, min_passenger, comfort_index_data)
        
    
# print "finish" once the for loop is finished
print("finish")

finish


# Read in taxi data (with covid), Merge with external data, Create new csv

In [9]:
def yellow_covid_process(date, initial_time, format_yellow, yellow_new_format, max_id, min_id, min_money, min_fare_total, min_distance, min_passenger, covid, comfort_index_data):
    '''
    This function is created to:
    Read in yellow taxi data
    
    Drop the useless columns (not related to tip_amount in the assumption)
    Drop the instances with irrational feature values (eg. trip distance < 0)
    
    Create column for recording each trip's duration (feature engineering with dropoff time - pickup time)
    Create columns to record daily covid count and comfort_index
    
    Merge the taxi data with external datasets (covid & comfort_index)
    
    Create the first_preprocessed csv files for further usage
    '''
    
    # set the initial timestap string for each read in month
    month_start = date + initial_time

    # read in the selected month data
    yellow_covid_data = pd.read_csv(format_yellow.format(year = date), index_col=False, delimiter = ',', header = 0, low_memory = False)
    
    # drop the columns which are assumed to be irrelavant to tip_amount (target)
    yellow_covid_data = yellow_covid_data.drop(columns = 'store_and_fwd_flag')
    yellow_covid_data = yellow_covid_data.drop(columns = 'RatecodeID')
    yellow_covid_data = yellow_covid_data.drop(columns = 'VendorID')

    # drop the irrational instance with .loc() method with listed conditions
    yellow_covid_data = yellow_covid_data.loc[(yellow_covid_data['passenger_count'] > min_passenger) &
                                              (yellow_covid_data['PULocationID'] <= max_id) &
                                              (yellow_covid_data['PULocationID'] >= min_id) &
                                              (yellow_covid_data['DOLocationID'] <= max_id) &
                                              (yellow_covid_data['DOLocationID'] >= min_id) &
                                              (yellow_covid_data['trip_distance'] > min_distance) &
                                              (yellow_covid_data['fare_amount'] >= min_fare_total) &
                                              (yellow_covid_data['extra'] >= min_money) &
                                              (yellow_covid_data['mta_tax'] >= min_money) &
                                              (yellow_covid_data['tip_amount'] >= min_money) &
                                              (yellow_covid_data['tolls_amount'] >= min_money) &
                                              (yellow_covid_data['improvement_surcharge'] >= min_money) &
                                              (yellow_covid_data['total_amount'] >= min_fare_total) &
                                              (yellow_covid_data['congestion_surcharge'] >= min_money)]
    
    
    # drop irrational instances base on pickup time (must be greater or equal to initial timestamp of this month)
    yellow_covid_data = yellow_covid_data.loc[(pd.to_datetime(yellow_covid_data['tpep_pickup_datetime']) >= datetime.strptime(month_start, '%Y-%m-%d %H:%M:%S'))]

    # create duration column to record trip's duration from the difference between dropoff and pickup time
    yellow_covid_data['duration'] = ((pd.to_datetime(yellow_covid_data.tpep_dropoff_datetime) - pd.to_datetime(yellow_covid_data.tpep_pickup_datetime)).dt.days * 60*24 + (pd.to_datetime(yellow_covid_data.tpep_dropoff_datetime) - pd.to_datetime(yellow_covid_data.tpep_pickup_datetime)).dt.seconds / 60).round(5)
    
    # drop irrational instances base on the difference between pickup (earlier) and dropoff (later) conditions
    yellow_covid_data = yellow_covid_data.loc[(yellow_covid_data['duration'] > 0)]
    
    # sort the dataframe with pickup time in ascend order
    yellow_covid_data = yellow_covid_data.sort_values(by = ['tpep_pickup_datetime'])

    
    # record the current month covid data for merging usage
    month_covid = []
    month_count = []
    for i in range(len(covid['DATE_OF_INTEREST'])):
        if date in covid['DATE_OF_INTEREST'][i]:
            month_covid.append(covid['DATE_OF_INTEREST'][i])
            month_count.append(covid['CASE_COUNT'][i])
    
    # record the current month comfort_index data for merging usage
    month_di = []
    month_value = []
    for i in range(len(comfort_index_data['date'])):
        if date in comfort_index_data['date'][i]:
            month_di.append(comfort_index_data['date'][i])
            month_value.append(comfort_index_data['DIcomfort_index'][i])

    # create list for external data (covid count & comfort_index) in this month along with taxi pickup time
    index = 0
    month_data = []
    month_comfort_index = []
    pickup_time = yellow_covid_data['tpep_pickup_datetime'].tolist()
    for i in range(len(pickup_time)):
        if (month_covid[index] in pickup_time[i]):
            month_data.append(month_count[index])
            month_comfort_index.append(month_value[index])
        else:
            if (index + 1 < len(month_covid)):
                index += 1
                month_data.append(month_count[index])
                month_comfort_index.append(month_value[index])
            else:
                month_data.append(-1)
                month_comfort_index.append(-9999)

                
    # merge the created covid column with the taxi data
    covid_column = pd.Series(month_data)
    yellow_covid_data['covid_count'] = covid_column.values
    
    # merge the created comfort_index column with the taxi data
    comfort_column = pd.Series(month_comfort_index)
    yellow_covid_data['comfort_index'] = comfort_column.values
    
    # remove irrational instances base on comfort_index and covid count data
    yellow_covid_data = yellow_covid_data.loc[(yellow_covid_data['comfort_index'] > -9999) & (yellow_covid_data['covid_count'] > 0)]

    # remove columns of pickup, dropoff columns
    yellow_covid_data = yellow_covid_data.drop(columns = 'tpep_pickup_datetime')
    yellow_covid_data = yellow_covid_data.drop(columns = 'tpep_dropoff_datetime')
    
    # create and store the first_processed csv data under folder 'first_preprocess' in 'raw_data' for futher usage
    yellow_covid_data.to_csv(yellow_new_format.format(year = date), index = False)

In [10]:
# Use for loop to read in have-covid taxi data and create first_preprocessed merged data for further usage
for i in exist_covid_date:
    yellow_covid_process(i, initial_time, yellow_format, yellow_new_format, max_id, min_id, min_money, min_fare_total, min_distance, min_passenger, covid_data, comfort_index_data)

# print "finish" once the for loop is finished
print("finish")

finish
